In [1]:
from keras.layers import Lambda, Input
from keras.models import Model

x = Input((1,))
y = Lambda(lambda x:x +1)(x)
m = Model(x,y)

yp = m.predict_on_batch([5,6,7])
print(yp)

Using TensorFlow backend.


[[ 6.]
 [ 7.]
 [ 8.]]


In [3]:
def kproc(x):
    return x**2 + 2*x +1

def kshape(input_shape):
    return input_shape

x = Input((1,))
y = Lambda(kproc,kshape)(x)
m = Model(x,y)
yp = m.predict_on_batch([5,6,7])
print(yp)

[[ 36.]
 [ 49.]
 [ 64.]]


In [4]:
from keras import backend as K

def kproc_concat(x):
    m = K.mean(x,axis=1,keepdims=True)
    d1 = K.abs(x-m)
    d2 = K.square(x-m)
    return K.concatenate([x,d1,d2],axis=1)

def kshape_concat(input_shape):
    output_shape = list(input_shape)
    output_shape[1] *=3
    return tuple(output_shape)

x = Input((3,))
y = Lambda(kproc_concat,kshape_concat)(x)
m = Model(x,y)

yp = m.predict_on_batch([[1,2,3],[3,4,6]])
print(yp)


[[ 1.          2.          3.          1.          0.          1.          1.
   0.          1.        ]
 [ 3.          4.          6.          1.33333349  0.33333349  1.66666651
   1.77777815  0.11111122  2.77777719]]


In [5]:
def mean_squared_error(y_true,y_pred):
    return K.mean(K.square(y_pred-y_true),axis=-1)

In [6]:
import tensorflow as tf

def kproc_concat(x):
    m = tf.reduce_mean(x,axis=1,keep_dims=True)
    d1 = tf.abs(x-m)
    d2 = tf.square(x-m)
    return tf.concat([x,d1,d2],axis=1)

def kshape_concat(input_shape):
    output_shape = list(input_shape)
    output_shape[1] *=3
    return tuple(output_shape)

x = Input((3,))
y = Lambda(kproc_concat,kshape_concat)(x)
m = Model(x,y)

yp = m.predict_on_batch([[1,2,3],[3,4,6]])
print(yp)

[[ 1.          2.          3.          1.          0.          1.          1.
   0.          1.        ]
 [ 3.          4.          6.          1.33333349  0.33333349  1.66666651
   1.77777815  0.11111122  2.77777719]]


In [17]:
import keras
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

from keras import initializers
igu = initializers.get('glorot_uniform')
iz = initializers.get('zeros')

class Sample(Layer):
    def __init__(self,No,**kwargs):
        self.No = No
        super().__init__(**kwargs)
        
    def build(self,inshape):
        self.w = self.add_weight("w",(inshape[1],self.No),initializer=igu)
        self.b = self.add_weight("b",(self.No,),initializer=iz)
        super().build(inshape)
        
    def call(self,x):
        return K.dot(x,self.w)+self.b
    
    def compute_output_shape(self,inshape):
        return (inshape[0],self.No)
    
x = np.array([0,1,2,3,4])
y = x*2+1

model = keras.models.Sequential()
model.add(Sample(1,input_shape=(1,)))
model.compile('SGD',mean_squared_error)
model.fit(x[:2],y[:2],epochs=1000,verbose=0)
print("Predictions:", model.predict(x[2:]).flatten())

Predictions: [ 4.96334887  6.93682814  8.91030693]


In [21]:
sess = tf.Session()
K.set_session(sess)
from keras.layers import Dense, Dropout
from keras.metrics import categorical_accuracy, categorical_crossentropy

class DNN():
    def __init__(self,Nin,Nh,Nout):
        self.X_ph = tf.placeholder(tf.float32,shape=(None,Nin))
        self.L_ph = tf.placeholder(tf.float32,shape=(None,Nout))
        
        H = Dense(Nh[0],activation='relu')(self.X_ph)
        H = Dropout(0.5)(H)
        H = Dense(Nh[1],activation='relu')(H)
        H = Dropout(0.25)(H)
        self.Y_tf = Dense(Nout,activation='softmax')(H)
        
        self.Loss_tf = tf.reduce_mean(categorical_crossentropy(self.L_ph,self.Y_tf))
        self.Train_tf = tf.train.AdamOptimizer().minimize(self.Loss_tf)
        self.Acc_tf = categorical_accuracy(self.L_ph,self.Y_tf)
        self.Init_tf = tf.global_variables_initializer()
        
from keras import datasets
from keras.utils import np_utils

def Data_func():
    (X_train,Y_train),(X_test,Y_test) = datasets.mnist.load_data()
    Y_train = np_utils.to_categorical(Y_train)
    Y_test = np_utils.to_categorical(Y_test)
    
    L,W,H = X_train.shape
    X_train = X_train.reshape(-1,W*H)
    X_test = X_test.reshape(-1,W*H)
    X_train = X_train/255
    X_test = X_test/255
    return (X_train,Y_train), (X_test,Y_test)

import matplotlib.pyplot as plt

def run(model,data,sess,epochs,batch_size = 100):
    (X_train,Y_train),(X_test,Y_test) = data
    sess.run(model.Init_tf)
    with sess.as_default():
        N_tr = X_train.shape[0]
        for epoch in range(epochs):
            for b in range(N_tr//batch_size):
                X_tr_b = X_train[batch_size*(b-1):batch_size*b]
                Y_tr_b = Y_train[batch_size*(b-1):batch_size*b]
                
                model.Train_tf.run(feed_dict={model.X_ph:X_tr_b,model.L_ph:Y_tr_b,K.learning_phase():1})
                
            loss = sess.run(model.Loss_tf,feed_dict={model.X_ph:X_test,model.L_ph:Y_test, K.learning_phase():0})
            acc = model.Acc_tf.eval(feed_dict={model.X_ph:X_test,model.L_ph:Y_test,K.learning_phase():0})
            print("Epoch {0}: loss = {1:.3f}, acc = {2:.3f}".format(epoch,loss,np.mean(acc)))
            
def main():
    Nin = 784
    Nh = [100,50]
    num_class =10
    
    data = Data_func()
    model = DNN(Nin,Nh,num_class)
    run(model,data,sess,10,100)
    
if __name__ == "__main__":
    main()

Epoch 0: loss = 0.224, acc = 0.933
Epoch 1: loss = 0.172, acc = 0.948
Epoch 2: loss = 0.149, acc = 0.956
Epoch 3: loss = 0.135, acc = 0.962
Epoch 4: loss = 0.128, acc = 0.963
Epoch 5: loss = 0.117, acc = 0.967
Epoch 6: loss = 0.118, acc = 0.966
Epoch 7: loss = 0.107, acc = 0.968
Epoch 8: loss = 0.104, acc = 0.970
Epoch 9: loss = 0.108, acc = 0.969
